In [ ]:
import sys
sys.path.append("../")
sys.path.append("../../../assets/diffusion")

# Get rid of annoying tf warning
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 



import numpy as np
np.set_printoptions(suppress=True)
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import bayesflow as beef
import pandas as pd
import pickle

from experiments import RandomWalkDiffusionExperiment
from models import StaticDiffusion, StationaryDiffusion, RandomWalkDiffusion, RegimeSwitchingDiffusion

In [ ]:
# If set to False, existing results will be loaded
# Set to True if you want to re-run the experiments
SIMULATE_DATA = True
TRAIN_NETWORKS = False

In [ ]:
random_walk_ddm = RandomWalkDiffusion()
experiment = RandomWalkDiffusionExperiment(random_walk_ddm)

## Simulation

In [ ]:
static_ddm = StaticDiffusion()
stationary_ddm = StationaryDiffusion()
# regime_switching_ddm = RegimeSwitchingDiffusion()

In [ ]:
if SIMULATE_DATA:
    static_ddm_sim = static_ddm.generate(200)
    stationary_ddm_sim = stationary_ddm.generate(200)
    random_walk_ddm_sim = random_walk_ddm.generate(200)
    # regime_switching_ddm_sim = regime_switching_ddm.generate(200)
    with open('../data/static_ddm_sim_200.pkl', 'wb') as f:
        pickle.dump(static_ddm_sim, f)
    with open('../data/stationary_ddm_sim_200.pkl', 'wb') as f:
        pickle.dump(stationary_ddm_sim, f)
    with open('../data/random_walk_ddm_sim_200.pkl', 'wb') as f:
        pickle.dump(random_walk_ddm_sim, f)
    # with open('../data/regime_switching_ddm_sim_200.pkl', 'wb') as f:
    #     pickle.dump(regime_switching_ddm_sim, f)
else:
    with open('../data/static_ddm_sim_200.pkl', 'rb') as f:
        static_ddm_sim = pickle.load(f)
    with open('../data/stationary_ddm_sim_200.pkl', 'rb') as f:
        stationary_ddm_sim = pickle.load(f)
    with open('../data/random_walk_ddm_sim_200.pkl', 'rb') as f:
        random_walk_ddm_sim = pickle.load(f)
    # with open('../data/regime_switching_ddm_sim_200.pkl', 'rb') as f:
    #     regime_switching_ddm_sim = pickle.load(f)
    

In [ ]:
static_ddm_sim.keys()

## Training

In [ ]:
if TRAIN_NETWORKS:
    history = experiment.run(
        epochs=50, 
        iterations_per_epoch=1000, 
        batch_size=8
    )
else:
    history = experiment.trainer.loss_history.get_plottable()

In [ ]:
f = beef.diagnostics.plot_losses(history)

## Evaluation